# Algebraic $E_\infty$-bialgebra on cubes

In this notebook we implement natural E_infinity bialgebra structures on the chains of standard cubes using the finitely presented E_infinity prop of [[1]](https://arxiv.org/abs/1808.00854). These natural bialgebras induce an E-infinity coalgebra structure on the chains of any cubical set.

## Preliminaries

### $E_\infty$-coalgebras and $E_\infty$-bialgebras

Let $k$ be a commutative and unital ring. In this notebook we will consider $k$ to be the field with two elements, but the following definitions and statements hold in general. 

An $E_\infty$-coalgebra structure on a chain complex $C$ is a morphism of operads 

$$\mathcal E \to \big\{ \mathrm{Hom} (C, C^{\otimes m}) \big\}_{m \geq 0}$$

where $\mathcal E(0) = k$ and $\mathcal E(m)$ is a free $k[\Sigma_m]$-module with the homology of a point. Such operad is called an $E_\infty$-operad.

An $E_\infty$-bialgebra structure on a chain complex $C$ is a morphism of props

$$\mathcal P \to \big\{ \mathrm{Hom} (C^{\otimes n}, C^{\otimes m}) \big\}_{n,m \geq 0}$$

that restricted to the operad $U(\mathcal P) = \{\mathcal P(1,m)\}_{m \geq 0}$ defines an $E_\infty$-coalgebra structure on $C$.

In [[1]](https://arxiv.org/abs/1808.00854) is shown that a choice of three maps: 

$$\Delta : C \to C \otimes C$$
of degree 0,
$$\varepsilon : C \to \mathbb F_2$$
of degree 0, and
$$\ast : C \otimes C \to C$$

of degree one, define an $E_\infty$-bialgebra structure on $C$ if they satisfy for every $x,y \in C$

$$(\partial \otimes \mathrm{id} + \mathrm{id} \otimes \partial) \Delta (x) = \Delta\,(\partial(x))$$

$$\partial \varepsilon (x) + \varepsilon(\partial x) = 0$$

$$\partial (x \ast y) + (\partial x) \ast y + x \ast (\partial y) 
= \varepsilon(x)y + \varepsilon(y)x$$
and 
$$(\varepsilon(x) \otimes \mathrm{id})\,\Delta(x) = x = (\mathrm{id} \otimes \varepsilon(x))\,\Delta(x)$$

$$\varepsilon(x \ast y) = 0.$$

In this notebook, we describe using this method a natural $E_\infty$-bialgebra structure on the chains of standard cubes. We remark that, using a standard technique from category theory, this defines an $E_\infty$-coalgebra structure on the chains of cubical sets.

### Chains on the standard cubes

Let $\mathrm I = [0,1]$ be the interval with its traditional cellular structure. We denote its cells by 

$$\mathrm{cells}(\mathrm I) = \big\{[0], [1], e\big\}.$$

A basis for $C_\bullet(\mathrm I^d;\;\mathbb F_2)$ is given by 
$$\mathrm{basis}(\mathrm I^d) = \big\{(x_i,\dots,x_d) \, :\, x_i \in \mathrm{cells}(\mathrm I) \big\}$$ 

and its chain complex structure is induced via the isomorphism

$$C_\bullet(\mathrm I^d) \cong C_\bullet(;\mathbb F_2)^{\otimes d}.$$

Some notation we will use in this notebook is the following: let $x = (x_1,\dots,x_d)$ and $y = (y_1,\dots,y_d)$

\begin{align*}
x_{<i} &= (x_1,\dots,x_{i-1}) \\
x_{>i} &= (x_{i+1},\dots,x_d) \\
\partial_i(x) &= (x_1, \dots, \partial x_i,\dots, x_n).
\end{align*}

## Main methods

### Basis, degree, and boundary

We introduce methods modeling the functions
$$d \mapsto \mathrm{basis}\big(C_\bullet(\mathrm I^d;\mathbb F_2)\big)$$

$$(x_1,\dots,x_d) \mapsto \mathrm{degree}\big((x_1,\dots,x_d)\big)$$

$$(x_1,\dots,x_d) \mapsto \partial(x_1,\dots,x_d).$$

In [1]:
from itertools import product as times

def basis(d):
    if d == 0:
        return ['0']
    
    cells = {'0','1','e'}
    basis = [''.join(ltr) for ltr in times(cells, repeat=d)]
    
    return basis

def degree(x):
    return x.count('e')

def boundary(x):
    answer = set()
    for i, letter in enumerate(x):
        if letter == 'e':
            answer ^= {x[:i]+'0'+x[i+1:],
                       x[:i]+'1'+x[i+1:]}
    return answer

def display_boundary(x):
    bdry = boundary(x)
    if not bool(bdry):
        return f'partial([{x}]) = 0'
    
    space = ' '*(12+len(x))
    bdry = str(bdry).replace("', '","]\n"+space+"+ [")
    return f'partial([{x}]) = ['+bdry[2:-2]+']'

#### Example

Given an choice of integer $d$, we print the boundary $\partial(x)$ for all basis elements in $C_\bullet(\mathrm I^d; \mathbb F_2)$.

In [2]:
d = 0

for x in basis(d):
    print(display_boundary(x))

partial([0]) = 0


### Coproduct

We will model the function

$$\Delta : C_\bullet(\mathrm I^d;\mathbb F_2) \to C_\bullet(\mathrm I^d;\mathbb F_2) \otimes C_{\bullet}(\mathrm I^d;\mathbb F_2)$$
defined by
$$ \Delta \big( (x_1,\dots x_d) \big) = 
(\pi_1\Delta(x_1), \dots, \pi_1\Delta(x_d)) \otimes
(\pi_2\Delta(x_1), \dots, \pi_2\Delta(x_d))$$
where 
$$\Delta([0]) = [0] \otimes [0] \qquad \Delta([1]) = [1] \otimes [1]$$
and
$$ \Delta(e) = e \otimes [0] + [1] \otimes e.$$

In [3]:
def coproduct(x, answer=set()):
    if len(x) == 0:
        return {('','')}
    
    if x[0] == 'e':        
        answer = {('0'+pair[0], 'e'+pair[1]) 
                  for pair in coproduct(x[1:], answer)} \
               ^ {('e'+pair[0], '1'+pair[1]) 
                  for pair in coproduct(x[1:], answer)}
    else:
        answer = {(x[0]+pair[0], x[0]+pair[1]) 
                  for pair in coproduct(x[1:], answer)}  
    return answer

def display_coproduct(x):
    coprod = str(coproduct(x))
    coprod = coprod.replace("', '","]|[")
    
    space = ' '*(10+len(x))
    coprod = coprod.replace("'), ('","]\n"+space+"+ [")
    
    return f'Delta([{x}]) = ['+coprod[3:-3]+']'

#### Example

Given an choice of integer $d$, we print the coproduct $\Delta(x)$ for all basis elements in $C_\bullet(\mathrm I^d; \mathbb F_2)$.

In [29]:
d = 1

for x in basis(d):
    print(display_coproduct(x))

Delta([1]) = [1]|[1]
Delta([0]) = [0]|[0]
Delta([e]) = [e]|[1]
           + [0]|[e]


### Counit

We will model the function 
$$\varepsilon : C_\bullet(\mathrm I^d;\mathbb F_2) \to \mathbb F_2$$ 
defined on basis elements by
$$\varepsilon(x) = 
\begin{cases}
1 & |x| = 0 \\
0 & |x| > 0.
\end{cases}$$

In [5]:
def counit(x):
    if degree(x) > 0:
        return 0
    if degree(x) == 0:
        return 1

#### Example
Given a choice of integer $d$, we print counit $\varepsilon(x)$ for all basis elements in $C_\bullet(\mathrm I^d; \mathbb F_2)$.

In [6]:
d = 0

for x in basis(d):
    print(f'varepsilon([{x}]) = ',counit(x))

varepsilon([0]) =  1


### Product

We will model the function
$$\ast : C_\bullet(\mathrm I^d;\mathbb F_2) \otimes C_\bullet(\mathrm I^d;\mathbb F_2) \to C_{\bullet+1}(\mathrm I^d;\mathbb F_2)$$
defined by
$$ \ast (x \otimes y) =
\sum_{i=1}^d \varepsilon(x_{<i})\, \varepsilon(y_{>i})\, \big( y_{<i}, (x_i\ast y_i), x_{>i} \big).$$
where $$\ast(x_i,y_i) = \begin{cases} 
e &  x_i=[0]\ y_i=[1] \text{ or} \\
  &  x_i=[1]\ y_i=[0] \\
0 & \text{otherwise}.
\end{cases}$$

In [7]:
_ast = {'01':'e','10':'e'}

def product(x,y):
    assert len(x)== len(y), 'not in the same cube'
    
    answer = set()
    for i, pair in enumerate(zip(x,y)):
        try:
            answer ^= {y[:i]
                      +_ast[pair[0]+pair[1]]
                      +x[i+1:]}
        except KeyError:
            pass
    
    expected_degree = degree(x)+degree(y)+1
    answer = {term for term in answer if 
              term.count('e') == expected_degree}
    
    return answer

def display_product(x,y):
    jn = product(x,y)
    
    if jn:
        return f'[{x}]*[{y}] = [' + ']+['.join(jn) + ']'
    if not jn:
        return f'[{x}]*[{y}] = 0'

#### Example

Given a choice of integer $d$, we print the product $x*y$ for all pairs of basis elements in $C_\bullet(\mathrm I^d; \mathbb F_2)$.

In [30]:
d = 1
only_nonzero = True

for x,y in times(basis(d), repeat=2):
    if product(x,y) or not only_nonzero:
        print(display_product(x,y))

[1]*[0] = [e]
[0]*[1] = [e]


## Checks and proofs

### Boundary of coproduct

We define a method that verifies that for any basis element $x \in C_\bullet(\mathrm I^d; \mathbb F_2)$

$$(\partial \otimes \mathrm{id} + \mathrm{id}\otimes \partial)\, \Delta(x) + \Delta(\partial x) = 0$$

In [9]:
def check_boundary_coproduct(x):
    '''returns True if the boundary the coproduct acting 
    on the basis element x is 0'''
    
    answer = set()
    for term in boundary(x):
        answer ^= coproduct(term)
        
    for pair in coproduct(x):
        bdry_0 = boundary(pair[0])
        answer ^= {(term, pair[1]) for term in bdry_0}
        
        bdry_1 = boundary(pair[1])
        answer ^= {(pair[0], term) for term in bdry_1}
        
    return not bool(answer)

### Boundary of the product

We define a method that verifies that for any pair of basis elements $x,y \in C_\bullet(\mathrm I^d; \mathbb F_2)$

$$\partial (x \ast y) + (\partial x) \ast y + x \ast (\partial y) 
= \varepsilon(x)y + \varepsilon(y)x.$$

In [ ]:
def check_product_boundary(x,y):
    '''returns True if the boundary of the product
    acts on a pair of basis elements x and y as
    counit(x)y + counit(y)x'''
    answer = set()
    for x_term in boundary(x):
        answer ^= product(x_term, y)
    for y_term in boundary(y):
        answer ^= product(x, y_term)
    for z in product(x,y):
        answer ^= boundary(z)
    if bool(counit(x)):
        answer ^= {y}
    if bool(counit(y)):
        answer ^= {x}

    return not bool(answer)

#### Proof

We have
\begin{align*}
\partial (x \ast y) &= 
\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( \partial y_{<j},\, x_j \ast y_j,\, x_{>j}\big) \\ &+
\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\ \partial(x_j \ast y_j),\ x_{>j} \big) \\ &+ 
\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\, x_j \ast y_j,\, \partial x_{>j}\big)
\Big),\\
\partial x \ast y &= 
\sum_{j=1}^n \sum_{i<j} \varepsilon(\partial_i x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\, x_j \ast y_j,\, x_{>j} \big) \\ & +
\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\, \partial x_j \ast y_j,\, x_{>j} \big) \\ & +
\sum_{j=1}^n \sum_{k>j} \varepsilon(x_{<j})\, \varepsilon(y_{>j})\,	\big( y_{<j},\, x_j \ast y_j,\, \partial_k x_{>j} \big), \\
x \ast \partial y &= 
\sum_{j=1}^n \sum_{i<j} \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( \partial_i y_{<j},\, x_j \ast y_j,\, x_{>j} \big) \\ & +
\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\, x_j \ast \partial y_j,\, x_{>j} \big) \\ & +
\sum_{j=1}^n \sum_{k>j} \varepsilon(x_{<j})\, \varepsilon(\partial_k y_{>j})\,	\big( y_{<j},\, x_j \ast y_j,\, x_{>j} \big).
\end{align*}

Using that $\varepsilon(\partial (z)) = 0$ for any $z$, we conclude that the left hand side is equal to 

\begin{align*}
&\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\, \partial(x_j \ast y_j),\, x_{>j} \big) \\ +
&\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\, \partial x_j \ast y_j,\, x_{>j} \big) \\ +
&\sum_{j=1}^n \varepsilon(x_{<j})\, \varepsilon(y_{>j})\, \big( y_{<j},\, x_j \ast \partial y_j,\, x_{>j} \big)
\end{align*}
which in turns, since 
\begin{equation*}
\partial (x_j \ast y_j) + \partial x_j \ast y_j + x_j \ast \partial y_j = \varepsilon(x_j) y_j + \varepsilon(y_j) x_j,
\end{equation*}
is also equal to 
\begin{equation*}
\sum_{j=1}^n \varepsilon(x_{\leq j})\, \varepsilon(y_{>j})\, \big( y_{\leq j},\, x_{>j} \big) + 
\varepsilon(x_{< j})\, \varepsilon(y_{\geq j})\, \big( y_{< j},\, x_{\geq j} \big).
\end{equation*}

Finally, a telescopic sum argument establishes that the left hand side is equal to

\begin{equation*}
\varepsilon(x)y + \varepsilon(y)x
\end{equation*}
as claimed.

### Counit property

We define a method that verifies that for any basis element $x \in C_\bullet(\mathrm I^d; \mathbb F_2)$

$$(\varepsilon(x) \otimes \mathrm{id})\,\Delta(x) = x = (\mathrm{id} \otimes \varepsilon(x))\,\Delta(x).$$

In [11]:
def check_counit_property(x):
    '''return True if applying the counit to any of the factors 
    of the coproduct of any basis element gives the element back'''
    coprod = coproduct(x)
    a = {pair[1] for pair in coprod if bool(counit(pair[0]))}
    a ^= {x}
    b = {pair[0] for pair in coprod if bool(counit(pair[1]))}    
    b ^= {x}
    return not (bool(a) or bool(b))

### Does not satisfy Leibniz

Take $[1] \otimes e$ to show it